In [1]:
import os

import numpy as np
from bokeh.layouts import column, row
from bokeh.models import CustomJS, Slider, Dropdown
from bokeh.plotting import ColumnDataSource, figure, show
from bokeh.util.compiler import JavaScript

from casatools import image

import NotebookTesting as nt

MPIEnvironment not Enabled


In [2]:
# Clean up files form last run
os.system('rm -rf standard_cube* *.png *.log *.html')

0

In [3]:
standard = nt.StandardTest('E2E6.1.00034.S_tclean.ms')

Finished CasaTest initialization.


In [4]:
standard.standard_cube()

standard_cube.iter
/export/home/ajax/jhoskins/Development/notebook/notebook-testing/standard_cube.iter1

STARTING: iter0 routine
Copying iter0 files to iter1
STARTING: iter1 routine
Image list:

/export/home/ajax/jhoskins/Development/notebook/notebook-testing/standard_cube.iter1
pstr after checkims = [ check_ims ] Image made : ['/export/home/ajax/jhoskins/Development/notebook/notebook-testing/standard_cube.iter1.psf', '/export/home/ajax/jhoskins/Development/notebook/notebook-testing/standard_cube.iter1.residual', '/export/home/ajax/jhoskins/Development/notebook/notebook-testing/standard_cube.iter1.image', '/export/home/ajax/jhoskins/Development/notebook/notebook-testing/standard_cube.iter1.image.pbcor', '/export/home/ajax/jhoskins/Development/notebook/notebook-testing/standard_cube.iter1.mask', '/export/home/ajax/jhoskins/Development/notebook/notebook-testing/standard_cube.iter1.pb', '/export/home/ajax/jhoskins/Development/notebook/notebook-testing/standard_cube.iter1.model', '/export/

--> Reports finished.


In [5]:
_ia = image()
_ia.open('standard_cube.iter1.image')

_ib = image()
_ib.open('standard_cube.iter1.residual')

True

In [6]:
_ia.history(list=False)

[]

In [7]:
cube = _ia.getchunk()
cube_residual = _ib.getchunk()

In [8]:
data = []
residual = []

for i in range(50,90):
    data.append({'d': [ cube[:,:,0,i].transpose()]})
    residual.append({'r': [ cube_residual[:,:,0,i].transpose()]})

_ia.close()
_ib.close()

True

In [9]:
source = ColumnDataSource(data=data[0])

fig = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
fig.x_range.range_padding = fig.y_range.range_padding = 0
fig.image(
    image="d", 
    x=0, 
    y=0, 
    dw=cube.shape[0], 
    dh=cube.shape[1], 
    palette="Spectral11", 
    level="image", 
    source=source )

fig.grid.grid_line_width = 0.5

In [10]:
residual_source = ColumnDataSource(data=residual[0])

fig_r = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")])
fig_r.x_range.range_padding = fig_r.y_range.range_padding = 0
fig_r.image(
    image="r", 
    x=0, 
    y=0, 
    dw=cube_residual.shape[0], 
    dh=cube_residual.shape[1], 
    palette="Spectral11", 
    level="image", 
    source=residual_source)

fig_r.grid.grid_line_width = 0.5

In [11]:
class WSSlider(Slider):

    __implementation__ = JavaScript(
        """
        import {Slider, SliderView} from "models/widgets/slider"
        export class WSSliderView extends SliderView {}
        WSSliderView.__name__ = "WSSliderView";
        
        export class WSSlider extends Slider {
            constructor(attrs) {
            super(attrs);
        
            this.websocket = new WebSocket("ws://127.0.0.1:6799/");
            this.websocket.onopen = () => this.websocket.send(JSON.stringify({action: 'established'}));
        }
        connect_signals() {
            super.connect_signals( );
            console.info(this)
            this.on_change([this.properties.value], 
                () => this.websocket.send(JSON.stringify({
                    action: 'channel', 
                    value: this.value}))
            )
        }}
            WSSlider.__name__ = "WSSlider";
        """ )

In [12]:
slider = WSSlider(start=0, end=len(data)-1, value=0, step=1, title="Channel")

#callback = CustomJS(
#    args=dict(
#        source=source,
#        data=data,
#        slider=slider), 
#    code="""
#    source.data = data[slider.value];
#    """)

#slider.js_on_change('value', callback)
slider.js_on_change("value", CustomJS(
    args=dict(
        source=source,
        source1=residual_source,
        data=data,
        data1=residual, 
        slider=slider), 
    code="""
    source.data = data[slider.value]
    source1.data = data1[slider.value]
    """))

layout = column(row(fig, fig_r), slider)
show(layout)